In [1]:
import psycopg2

**Справочники по объектам**

*МФЦ*

In [8]:
with psycopg2.connect("host=65.21.50.30 dbname=postgis port=25432 user=admin password=admin") as conn:
    with conn.cursor() as cursor:
        sql = """
    DROP TABLE IF EXISTS public.mfc_info;

    WITH mfc AS 
    ( 
      SELECT distinct 
          name, point_lat, point_lon, address_name, 
          ST_SetSRID(ST_point(point_lon, point_lat), 4326) as point_geo
      from public.fixed_points a
      where 1 = 1
    -- ------- МФЦ
           AND type NOT IN ('attraction', 'station', 'street', 'parking', 'adm_div', 'building')
           AND lower(name) LIKE '%мои документы, центр государственных услуг%'
    
    )
    , find_zid AS 
    (
      SELECT mfc.*
        , zid
        , ST_SetSRID(center, 4326) center
        , ST_SetSRID(geometry_base, 4326) as geometry_base
        , ST_SetSRID(pol_15min, 4326) as pol_15min
      FROM mfc
      LEFT JOIN public.izochrones_by_walk a
        ON ST_Contains(ST_SetSRID(geometry_base, 4326), ST_SetSRID(point_geo, 4326))
      WHERE geometry_base is NOT NULL
    )
    , adm_zones AS 
    (
      SELECT cell_zid
        , MAX(adm_name) as adm_name
        , MAX(okrug_name) AS okrug_name
        , MAX(sub_ter) sub_ter
      FROM public.adm_regions 
      GROUP BY cell_zid
    )
    , all_areas as 
    (
        select  i.izid izid, e.zid ezid, i.customers_cnt_home
        from optimizer.entity e join optimizer.point_in_polygons_intersections i
                on e.zid = i.ezid
        where kind = 'mfc'
    ) 
    , porportional_cnt_home as (
        select izid,
               sum(customers_cnt_home) sum_customers_cnt_home,
               count(distinct ezid) countd_izid,
               customers_cnt_home / count(distinct ezid) prop_home,
               array_agg(distinct ezid) ezid_list
        from all_areas
        group by izid, customers_cnt_home
    )
    , right_population AS
    (
      select ezid, sum(prop_home) population
      from (select unnest(ezid_list) ezid, prop_home from porportional_cnt_home) q
      group by 1
    )
    SELECT t1.*
        , t2.adm_name
        , t2.okrug_name
        , t2.sub_ter
        , t3.population
        
    INTO public.mfc_info
    
    FROM find_zid t1
    LEFT JOIN adm_zones t2
      ON t1.zid = t2.cell_zid
    LEFT JOIN right_population t3 
      ON t1.zid = t3.ezid
    """
        cursor.execute(sql)
        conn.commit()

*Школы*

In [9]:
with psycopg2.connect("host=65.21.50.30 dbname=postgis port=25432 user=admin password=admin") as conn:
    with conn.cursor() as cursor:
        sql = """
    DROP TABLE IF EXISTS public.school_info;

    WITH mfc AS 
    ( 
      SELECT distinct 
          name, point_lat, point_lon, address_name, 
          ST_SetSRID(ST_point(point_lon, point_lat), 4326) as point_geo
      from public.fixed_points a
      where 1 = 1
    -------ШКОЛЫ
           AND type NOT IN ('attraction', 'station', 'street', 'parking', 'adm_div', 'building')
           AND  (lower(name) LIKE '%лицей%' OR lower(name) LIKE '%гимназия%' OR lower(name) LIKE '%общеобразовательная%школа%'
               OR lower(name) LIKE '%школа%№%' OR lower(name) LIKE '%школа%дошкольн%')
           AND lower(name) NOT LIKE '%магазин%'
           AND lower(name) NOT LIKE '%журнал%'
           AND lower(name) NOT LIKE '%ветеринар%'
           AND lower(name) NOT LIKE '%автошкола%'
           AND lower(name) NOT LIKE '%автолицей%' 
           AND lower(name) NOT LIKE '%спортивная%' 
    
    )
    , find_zid AS 
    (
      SELECT mfc.*
        , zid
        , ST_SetSRID(center, 4326) center
        , ST_SetSRID(geometry_base, 4326) as geometry_base
        , ST_SetSRID(pol_15min, 4326) as pol_15min
      FROM mfc
      LEFT JOIN public.izochrones_by_walk a
        ON ST_Contains(ST_SetSRID(geometry_base, 4326), ST_SetSRID(point_geo, 4326))
      WHERE geometry_base is NOT NULL
    )
    , adm_zones AS 
    (
      SELECT cell_zid
        , MAX(adm_name) as adm_name
        , MAX(okrug_name) AS okrug_name
        , MAX(sub_ter) sub_ter
      FROM public.adm_regions 
      GROUP BY cell_zid
    )
    , all_areas as 
    (
        select  i.izid izid, e.zid ezid, i.customers_cnt_home
        from optimizer.entity e join optimizer.point_in_polygons_intersections i
                on e.zid = i.ezid
        where kind = 'school'
    ) 
    , porportional_cnt_home as (
        select izid,
               sum(customers_cnt_home) sum_customers_cnt_home,
               count(distinct ezid) countd_izid,
               customers_cnt_home / count(distinct ezid) prop_home,
               array_agg(distinct ezid) ezid_list
        from all_areas
        group by izid, customers_cnt_home
    )
    , right_population AS
    (
      select ezid, sum(prop_home) population
      from (select unnest(ezid_list) ezid, prop_home from porportional_cnt_home) q
      group by 1
    )
    SELECT t1.*
        , t2.adm_name
        , t2.okrug_name
        , t2.sub_ter
        , t3.population
        
    INTO public.school_info
    
    FROM find_zid t1
    LEFT JOIN adm_zones t2
      ON t1.zid = t2.cell_zid
    LEFT JOIN right_population t3 
      ON t1.zid = t3.ezid
    """
        cursor.execute(sql)
        conn.commit()

*Детские сады*

In [10]:
with psycopg2.connect("host=65.21.50.30 dbname=postgis port=25432 user=admin password=admin") as conn:
    with conn.cursor() as cursor:
        sql = """
    DROP TABLE IF EXISTS public.kindergarden_info;

    WITH mfc AS 
    ( 
      SELECT distinct 
          name, point_lat, point_lon, address_name, 
          ST_SetSRID(ST_point(point_lon, point_lat), 4326) as point_geo
      from public.fixed_points a
      where 1 = 1
    ------ДЕТ САДЫ  
       AND type NOT IN ('attraction', 'station', 'street', 'parking', 'adm_div', 'building')
       AND  (lower(name) LIKE '%ясли%' OR lower(name) LIKE '%детский%сад%')
       AND lower(name) NOT LIKE '%магазин%'
       AND lower(name) NOT LIKE '%журнал%'
    
    )
    , find_zid AS 
    (
      SELECT mfc.*
        , zid
        , ST_SetSRID(center, 4326) center
        , ST_SetSRID(geometry_base, 4326) as geometry_base
        , ST_SetSRID(pol_15min, 4326) as pol_15min
      FROM mfc
      LEFT JOIN public.izochrones_by_walk a
        ON ST_Contains(ST_SetSRID(geometry_base, 4326), ST_SetSRID(point_geo, 4326))
      WHERE geometry_base is NOT NULL
    )
    , adm_zones AS 
    (
      SELECT cell_zid
        , MAX(adm_name) as adm_name
        , MAX(okrug_name) AS okrug_name
        , MAX(sub_ter) sub_ter
      FROM public.adm_regions 
      GROUP BY cell_zid
    )
    , all_areas as 
    (
        select  i.izid izid, e.zid ezid, i.customers_cnt_home
        from optimizer.entity e join optimizer.point_in_polygons_intersections i
                on e.zid = i.ezid
        where kind = 'kindergarden'
    ) 
    , porportional_cnt_home as (
        select izid,
               sum(customers_cnt_home) sum_customers_cnt_home,
               count(distinct ezid) countd_izid,
               customers_cnt_home / count(distinct ezid) prop_home,
               array_agg(distinct ezid) ezid_list
        from all_areas
        group by izid, customers_cnt_home
    )
    , right_population AS
    (
      select ezid, sum(prop_home) population
      from (select unnest(ezid_list) ezid, prop_home from porportional_cnt_home) q
      group by 1
    )
    SELECT t1.*
        , t2.adm_name
        , t2.okrug_name
        , t2.sub_ter
        , t3.population
        
    INTO public.kindergarden_info
    
    FROM find_zid t1
    LEFT JOIN adm_zones t2
      ON t1.zid = t2.cell_zid
    LEFT JOIN right_population t3 
      ON t1.zid = t3.ezid
    """
        cursor.execute(sql)
        conn.commit()

*Клиники*

In [11]:
with psycopg2.connect("host=65.21.50.30 dbname=postgis port=25432 user=admin password=admin") as conn:
    with conn.cursor() as cursor:
        sql = """
    DROP TABLE IF EXISTS public.clinic_info1;

    WITH mfc AS 
    ( 
      SELECT distinct 
          name, point_lat, point_lon, address_name, 
          ST_SetSRID(ST_point(point_lon, point_lat), 4326) as point_geo
      from public.fixed_points a
      where 1 = 1
    ------ ПОЛИКЛИНИКА
      AND type NOT IN ('attraction', 'station', 'street', 'parking', 'adm_div', 'building')
      AND  (lower(name) LIKE '%поликлиника%' --OR lower(name) LIKE '%медицинский%центр%' -- lower(name) LIKE '%клиника%' OR 
          OR lower(name) LIKE '%больница%')
      AND lower(name) NOT LIKE '%мама и малыш%'    
      AND lower(name) NOT LIKE '%ветеринар%'
      AND lower(name) NOT LIKE '%медросконтракт%'
      AND lower(name) NOT LIKE '%медсемья%'
      AND lower(name) NOT LIKE '%косметолог%' 
      AND lower(name) NOT LIKE '%нефросовет%'
      AND name NOT LIKE '%ПолиКлиника%'
      AND lower(name) NOT LIKE '%поликлиника.ру%'
      AND lower(name) NOT LIKE '%аптечный%'
      AND lower(name) NOT LIKE '%салон%оптики%'
      AND lower(name) NOT LIKE '%молочно%раздаточн%'
      AND lower(name) NOT LIKE '%журнал%'
      AND lower(name) NOT LIKE '%автобус%'
      AND lower(name) NOT LIKE '%автобус%'
    
    )
    , find_zid AS 
    (
      SELECT mfc.*
        , zid
        , ST_SetSRID(center, 4326) center
        , ST_SetSRID(geometry_base, 4326) as geometry_base
        , ST_SetSRID(pol_15min, 4326) as pol_15min
      FROM mfc
      LEFT JOIN public.izochrones_by_walk a
        ON ST_Contains(ST_SetSRID(geometry_base, 4326), ST_SetSRID(point_geo, 4326))
      WHERE geometry_base is NOT NULL
    )
    , adm_zones AS 
    (
      SELECT cell_zid
        , MAX(adm_name) as adm_name
        , MAX(okrug_name) AS okrug_name
        , MAX(sub_ter) sub_ter
      FROM public.adm_regions 
      GROUP BY cell_zid
    )
    , all_areas as 
    (
        select  i.izid izid, e.zid ezid, i.customers_cnt_home
        from optimizer.entity e join optimizer.point_in_polygons_intersections i
                on e.zid = i.ezid
        where kind = 'clinic'
    ) 
    , porportional_cnt_home as (
        select izid,
               sum(customers_cnt_home) sum_customers_cnt_home,
               count(distinct ezid) countd_izid,
               customers_cnt_home / count(distinct ezid) prop_home,
               array_agg(distinct ezid) ezid_list
        from all_areas
        group by izid, customers_cnt_home
    )
    , right_population AS
    (
      select ezid, sum(prop_home) population
      from (select unnest(ezid_list) ezid, prop_home from porportional_cnt_home) q
      group by 1
    )
    SELECT t1.*
        , t2.adm_name
        , t2.okrug_name
        , t2.sub_ter
        , t3.population
        
    INTO public.clinic_info1
    
    FROM find_zid t1
    LEFT JOIN adm_zones t2
      ON t1.zid = t2.cell_zid
    LEFT JOIN right_population t3 
      ON t1.zid = t3.ezid
    """
        cursor.execute(sql)
        conn.commit()